#IRE Team Project
Project Title: Auto Annotaion of document


# Prepare Dataset for training

In [1]:
!pip install -q kaggle

Upload kaggle.json file to colab, which can be download from Account->api->download 

In [2]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ./kaggle.json

In [3]:
!ls

kaggle.json  sample_data


In [4]:
# download kaggle api json file add it to vm 
!kaggle datasets download -d aashita/nyt-comments

 99% 473M/480M [00:02<00:00, 251MB/s]
100% 480M/480M [00:03<00:00, 166MB/s]


> Articles*.csv contains the keywords which will be later used in GPT2 for conditional commne genration process.

> Commnets*.csv conatins the comment on article with respective Article-ID

In [5]:
!unzip nyt-comments.zip

Archive:  nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [6]:
import pandas as pd
dataframe = pd.Series()
dataframe

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


Series([], dtype: float64)

In [7]:
# import os
# arr = os.listdir('.')
# import re
# res=[]
# for i in arr:
#   x = re.search("^Comments*", i)
#   if x: res.append(i)
# # res
# for i in res:
#   x=pd.read_csv(i)
#   # print(x['commentBody'])
#   dataframe=dataframe.append(x['commentBody'],ignore_index=True) 
#   print(dataframe.describe())

In [8]:
x=pd.read_csv('CommentsMarch2018.csv')
y=pd.read_csv('CommentsApril2018.csv')
# print(x['commentBody'])
dataframe=dataframe.append(x['commentBody'],ignore_index=True)
# dataframe=dataframe.append(y['commentBody'],ignore_index=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is 
lightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the !nvidia-smi

In [9]:
%tensorflow_version 1.x
!pip install git+https://github.com/rewrihitesh/gpt-2-simple.git
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow as tf
!nvidia-smi

TensorFlow 1.x selected.
  Cloning https://github.com/rewrihitesh/gpt-2-simple.git to /tmp/pip-req-build-icq22uii
  Running command git clone -q https://github.com/rewrihitesh/gpt-2-simple.git /tmp/pip-req-build-icq22uii
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=24457 sha256=b3a1866e9458427095d6b5e553eb9350c7467c1f4e09a1bafa69295ebe5ea71d
  Stored in directory: /tmp/pip-ephem-wheel-cache-jikr7k45/wheels/f3/60/c4/4bc4e025c301fcd75f4a55bb039ee0d8948b5fa28ccacd71b8
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Thu Oct 29 06:41:29 2020       
+-------------------------------------------------------------

## Downloading GPT-2

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.

* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it or we can move it to gdrive.

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell will mount your personal Google Drive in the VM, which later cells can use to get data in/out.

In [10]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [11]:
# restart the run time 
% cd drive/My\ Drive/Colab-Notebooks/ire_final_project

/content/drive/My Drive/Colab-Notebooks/ire_final_project


In [12]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 88.3Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 174Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:11, 44.2Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 241Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 116Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 125Mit/s]                                                       


In [13]:
df=dataframe.to_frame('commentBody')

In [14]:
df.to_csv('march18.csv',index=False)

In [15]:
!ls

apr17.csv  checkpoint  march18_april18.csv  march18.csv  models  samples


In [16]:
file_name = "march18_april18.csv"

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
tf.reset_default_graph()
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=1000,
              restore_from='fresh',
              run_name='march18_april18',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:06<00:00,  6.13s/it]


In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='march18_april18')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='march18_april18')

# Genrating text and Finetuning

> Un-conditional Text Genration

GPT2 will genrate the text on its own.

> Conditional Text Genration

We can pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there.


You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

* **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=100,
              temperature=0.8,
              top_k=40,
              top_p=0.9,
              # prefix="<|startoftext|>",
              prefix="Democratic",
              truncate="<|endoftext|>",
              include_prefix=True,
              nsamples=5,
              batch_size=5
              )